In [ ]:
# leadership = CourseType.objects.get_or_create(
#     name="leadership",
#     name_fa="پیش‌وایی",
#     category=1
# )[0]
# relationship = CourseType.objects.get_or_create(
#     name="relationship",
#     name_fa="رابطه",
#     category=1
# )[0]
# pileh = CourseType.objects.get_or_create(
#     name="pileh",
#     name_fa="پیله",
#     category=1
# )[0]
# heart_of_matter = CourseType.objects.get_or_create(
#     name="heart of matter",
#     name_fa="جان کلام",
#     category=2
# )[0]
# accomplishment = CourseType.objects.get_or_create(
#     name="accomplishment",
#     name_fa="دستاورد",
#     category=2
# )[0]
# confront = CourseType.objects.get_or_create(
#     name="confront",
#     name_fa="مواجهه",
#     category=2
# )[0]
# transforming_year = CourseType.objects.get_or_create(
#     name="transforming year",
#     name_fa="تحویل سال",
#     category=2
# )[0]

# course_to_create = []
# course_to_create = course_to_create + [(leadership, i+1) for i in range(20)]
# course_to_create = course_to_create + [(relationship, i+1) for i in range(5)]
# course_to_create = course_to_create + [(pileh, i+1) for i in range(6)]
# course_to_create = course_to_create + [(heart_of_matter, i+1) for i in range(3)]
# course_to_create = course_to_create + [(accomplishment, i+1) for i in range(1)]
# course_to_create = course_to_create + [(confront, i+1) for i in range(3)]
# course_to_create = course_to_create + [(transforming_year, i+1) for i in range(3)]
# done_courses = Course.objects.bulk_create(
#     Course(
#         course_type=course_type,
#         number = i,
#         course_name=f"{course_type} {i}"
#     ) for course_type, i in course_to_create
# )

In [ ]:
from tqdm import tqdm
import pandas as pd
from commons.utils import convert_to_english_digit, normalize_phone

df = pd.read_excel("notebooks/course info.xlsx", sheet_name="تجربه‌گرها")

# df['normalized_phone'] = df['fixed phone'].apply(normalize_phone)
# good_phones = df['normalized_phone'].dropna().unique()

good_phone = []
bad_phone = []
for phone in df["fixed phone"]:
    if len(str(phone).strip()) == 11 and str(phone).startswith("09"):
        good_phone.append("+989" + phone[2:])
    else:
        bad_phone.append(phone)
good_phone = list(set(good_phone))
bad_phone = list(set(bad_phone))

users_dic = {user.phone_number: user for user in User.objects.filter(phone_number__in=good_phone)}


def get_gender(gender):
    if gender in ["M", "m", "مرد"]:
        return 1
    elif gender in ["F", "f", "زن"]:
        return 2
    return None


def get_national_id(national_id):
    if not national_id:
        return None
    if len(str(national_id).strip()) == 8:
        return "00" + convert_to_english(str(national_id)).strip()
    return convert_to_english(str(national_id)).strip()


def get_status(status):
    if status in ["انتقال به دوره های بعد", "انصراف قبل از دوره", "غایب", "انصراف پیش از دوره"]:
        return 2, 3
    elif status in ["انصراف روز اول", "انتخاب نماندن"]:
        return 3, 3
    elif status in ["انصراف روز اول - غایب", "ترک دوره"]:
        return 4, 3
    elif status in ["تیم اجرایی"]:
        return 7, 3
    elif status in ["مهمان"]:
        return 8, 3
    elif status in ["شرکت کننده", "حاضر در دوره"]:
        return 9, 3
    elif status in ["ثبت‌نام", "در انتظار پرداخت"]:
        return 1, 3
    elif status in ["حاضر در دوره - انتظار پرداخت"]:
        return 9, 2
    elif status in ["حاضر در دوره - عدم سررسید"]:
        return 9, 1
    elif status in ["حاضر در دوره - تسویه"]:
        return 9, 3
    return 9, 3


done_registrations = Registration.objects.values_list("user_id", "course_id")
registration_dic = {f"{reg[0]}-{reg[1]}": True for reg in done_registrations}
course_dic = {
    f"{course.course_type.name_fa}-{course.number}": course
    for course in Course.objects.select_related("course_type").all()
}

bad_data = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    phone = normalize_phone(row["fixed phone"])
    first_name = str(row.get("نام", "")).strip()
    last_name = str(row.get("نام‌خانوادگی", "")).strip()
    national_id = get_national_id(row.get("کد ملی", ""))
    if len(national_id) > 10:
        print("Invalid national ID", index, phone, national_id)
        national_id = None
    if not phone:
        continue
    if phone in users_dic:
        user = users_dic[phone]
        if user.first_name != first_name or user.last_name != last_name:
            bad_data.append((index, phone, user.full_name, first_name, last_name))
            continue
    else:
        user = User.objects.create(
            username=phone,
            phone_number=phone,
            first_name=first_name,
            last_name=last_name,
            telegram_id=row.get("تلگرام", None) or row.get("آی‌دی تلگرام", None),
            email=row.get("ایمیل", ""),
            profession=row.get("حرفه", ""),
            age=row.get("سن", None),
            gender=get_gender(row.get("جنسیت", None)),
            national_id=national_id,
            english_first_name=row.get("Name", ""),
            english_last_name=row.get("Surname", ""),
            referer_name=row.get("معرف", None),
        )
        users_dic[user.phone_number] = user
    course_name = row["نام دوره"]
    course_number = int(row["شماره دوره"])
    course = course_dic.get(f"{course_name}-{course_number}", None)
    if not course:
        print(f"Course not found: {course_name} - {course_number}", index, phone)
        continue
    status, payment_status = get_status(row.get("وضعیت", None))
    tuition = row.get("مبلغ نهایی", 0)
    if registration_dic.get(f"{user.id}-{course.id}", None) is not None:
        continue
    try:
        user = Registration.objects.create(
            user=user,
            course=course,
            status=status,
            payment_status=payment_status,
            tuition=tuition,
        )
        registration_dic[f"{user.id}-{course.id}"] = True
    except:
        print("Error", phone, user.full_name, row.get("نام"))

print("Done", len(bad_data), "bad data entries")

100%|█████████████████████████████████████████████████████████████████████████████████████████| 2048/2048 [00:00<00:00, 3011.87it/s]

Done 51 bad data entries


In [2]:
bad_data

[(170, '+989394310605', 'مارال احمدی', 'آرزو', 'نصیری'),
 (200, '+989122091903', 'محمد علی صابری', 'محمدعلی', 'صابری'),
 (214, '+989203109686', 'محمدعلى طائبى', 'محمدعلی', 'طائبی'),
 (239, '+989123441962', 'امیرعلی حسینخان', 'امیرعلی', 'حسین\u200cخان'),
 (244, '+989390372267', 'آرش آل بیگی غلامی', 'آرش', 'آل\u200cبیگی'),
 (250, '+989366205771', 'سعید عالم ورزنه اصفهانی', 'سعید', 'عالم'),
 (269, '+989194203646', 'بهنوش حسینی', 'سیده زینب', 'حسینی'),
 (283, '+989124386498', 'یاسمن بقینی', 'یاسمن', 'یقینی'),
 (339, '+989128300124', 'سید هانی امام جمعه', 'هانی', 'امام جمعه'),
 (379, '+989131798395', 'محمد ملکشاهی', 'رعنا', 'نظیفی'),
 (382, '+989355408007', 'مصطفی حسامی', 'مصطفی', 'حصامی'),
 (393, '+989121155875', 'علی اقدم', 'محمد', 'سبحانی'),
 (506, '+989195258134', '', 'نیما', 'بهرنگ'),
 (518, '+989120555019', 'امیررضا فتحی', 'محمودرضا', 'فتحی'),
 (529, '+989339085993', 'محمد مهدی زاده', 'سجاد', 'مغانلو'),
 (591, '+989331991386', 'شایان شریفی', 'امیررضا', 'فتحی'),
 (609, '+989366205771',

In [1]:
import random
import string

support_team = [
    ["احمد", "محمدی لیری", "+989163918667"],
    ["حسین", "سلمان", "+989367759388"],
    ["رویا", "مرادی‌زاده", "+989903283583"],
    ["سیدمحمد", "سیارپور", "+989125215405"],
    ["مریم", "جاوید", "+989126261105"],
    ["شهاب", "آذرکسب", "+989939194985"],
    ["زهرا", "یارایی شهمیرزادی", "+989356815558"],
    ["امیر", "کرمانشاهانی", "+989011063032"],
    ["ائل‌نور", "نقی‌نژاد", "+989149151966"],
    ["محمود", "کاشف", "+989103000558"],
    ["مرضیه", "خدادادی", "+989012572179"],
    ["علی", "آرمیده", "+989932942418"],
    ["فرزانه", "فیروز", "+989195061290"],
    ["حمیدرضا", "حر", "+989367704992"],
    ["صدف", "علیدوست", "+989366314477"],
    ["فرزاد", "سلطان آبادی", "+989125936532"],
    ["مرجان", "ناقور", "+989122208940"],
    ["مهسا", "سبزی", "+989102722402"],
    ["امیرمحمد", "حریمی", "+989301411201"],
    ["دیبا", "یکتا", "+989140020678"],
]
data = []
for first_name, last_name, phone in support_team:
    if not User.objects.filter(phone_number=phone).exists():
        user = User.objects.create(
            phone_number=phone,
            username=phone,
            first_name=first_name,
            last_name=last_name,
            is_staff=True,
            is_superuser=False,
        )
        random_password = "".join(random.choices(string.ascii_letters + string.digits, k=10))
        user.set_password(random_password)
        data.append([first_name, last_name, phone, random_password])
        print(first_name, last_name, phone, random_password)
    elif User.objects.filter(phone_number=phone, is_staff=False).exists():
        user = User.objects.get(phone_number=phone)
        user.is_staff = True
        user.save()
        random_password = "".join(random.choices(string.ascii_letters, k=12))
        user.set_password(random_password)
        data.append([first_name, last_name, phone, random_password])
        print(first_name, last_name, phone, random_password)

احمد محمدی لیری +989163918667 sigtopELEQFT
حسین سلمان +989367759388 gY5j5NoDak
رویا مرادی‌زاده +989903283583 NSM5tmS5YO
سیدمحمد سیارپور +989125215405 Q6zmPQUGGP
مریم جاوید +989126261105 HJrCHKfeZnWF
شهاب آذرکسب +989939194985 ujTc1n03sI
زهرا یارایی شهمیرزادی +989356815558 MRUOAaxDlX
امیر کرمانشاهانی +989011063032 O7vTxxmCXV
ائل‌نور نقی‌نژاد +989149151966 vpAGnayuNN
محمود کاشف +989103000558 WKTlYqRrskxi
مرضیه خدادادی +989012572179 s30r0ogJ9L
علی آرمیده +989932942418 GSTHjJ0uZ9
فرزانه فیروز +989195061290 uEZSIfwQBkFV
حمیدرضا حر +989367704992 ZcpINdBerJnK
صدف علیدوست +989366314477 47FnIbPo3f
فرزاد سلطان آبادی +989125936532 zepORXwZlwsR
مرجان ناقور +989122208940 oTnYbJkWhNiL
مهسا سبزی +989102722402 QcZ4lIXO1C
امیرمحمد حریمی +989301411201 lzWjGEWjGPMo
دیبا یکتا +989140020678 8Q7VIaS4Uf


In [ ]:
import pandas as pd
from tqdm import tqdm
import re
from commons.utils import convert_to_english_digit, normalize_phone


def extract_text(text, default=None):
    if text is None or str(text) in ["nan", "Nan", "NaN"]:
        return default
    return str(text)


staff_map = {
    "احمد": "+989163918667",
    "حسین": "+989367759388",
    "فرزاد": "+989125936532",
    "رویا": "+989903283583",
    "سیدمحمد": "+989125215405",
    "ماری": "+989126261105",
    "شهاب": "+989939194985",
    "زهرا": "+989356815558",
    "امیر": "+989011063032",
    "ائل‌نور": "+989149151966",
    "محمود": "+989103000558",
    "مرضیه": "+989012572179",
    "علی": "+989932942418",
    "فرزانه": "+989195061290",
    "حمیدرضا حر": "+989367704992",
    "صدف": "+989366314477",
    "مرجان": "+989122208940",
    "مهسا سبزی": "+989102722402",
    "امیرمحمد حریمی": "+989301411201",
    "دیبا": "+989140020678",
}
staff_map_user = {n: User.objects.get(phone_number=phone) for n, phone in staff_map.items()}


not_found = []
no_support = []
correct = 0
added = 0

dfs = [pd.read_excel("notebooks/call.xlsx", sheet_name=f"Team {i + 1}", dtype=str) for i in range(4)]
for df in dfs:
    df["phone"] = df["شماره تماس"].apply(normalize_phone)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        phone = row["phone"]
        if phone is None or str(phone) in ["nan", "Nan", "NaN"]:
            continue
        if (
            row.get("هم‌یار") is None
            or str(row.get("هم‌یار")) in ["nan", "Nan", "NaN"]
            or staff_map_user.get(row.get("هم‌یار", "").strip()) is None
        ):
            no_support.append((index, phone, row.get("هم‌یار")))
            continue
        support = staff_map_user.get(row.get("هم‌یار", "").strip())
        if phone is not None and phone != "" and phone != "0" and phone != "0":
            if User.objects.filter(phone_number=phone).count() == 0:
                not_found.append((index, phone))
                continue
            correct += 1
            user = User.objects.get(phone_number=phone)
            crm = user.crm_user
            crm.supporting_user = support
            crm.save()
            answer = row.get("پاسخ داد؟")
            desc = extract_text(row.get("توضیحات"), "").strip()
            crs = extract_text(row.get("دوره‌هایی که بوده"), "")
            exp = extract_text(row.get("تجربه‌ش از دوره"), "")
            how = extract_text(row.get("حالش چطوره؟"), "")
            description = (((desc + "\n" + crs).strip() + "\n" + exp).strip() + "\n" + how).strip()
            if CrmLog.objects.filter(crm=crm, user=support, action=1).exists():
                continue
            if len(description) > 1 or str(answer) == "True":
                added += 1
                crm_log = CrmLog.objects.create(crm=crm, user=support, action=1, description=description)
print(len(not_found), "not found", len(no_support), "no support", added, "added", correct, "correct")

100%|████████████████████████████████████████████████████████████████████████████████████████████| 999/999 [00:06<00:00, 147.40it/s]

0 not found 51 no support 1 added 255 correct


In [3]:
from django.contrib.auth.models import Group, Permission

code_names_support = [
    "view_crmlog",
    "add_crmlog",
    "change_crmlog",
    "view_crmuser",
    "change_crmuser",
    "view_registration",
    "change_registration",
    "view_user",
]
supporting_group, created = Group.objects.get_or_create(name="supporting")
for code_name in code_names_support:
    perm = Permission.objects.get(codename=code_name)
    supporting_group.permissions.add(perm)